# Semantic Segmentation

### Problem Statement

You are given an image of dimension $H$ by $W$, and a set of labels (eg, `cat, tree, sky`) ($\| \text{classes}\| = C$). With Semantic Segmentation, you are expected to label each of the $H \cdot W$ pixels with the appropriate class.

For instance
```
(image)               (pixel labels)
[[0.23, 0.05], ----> [[0, 1],
 [0.15, 0.03]]        [0, 1]]
```

### A Naive Approach
A Naive approach might be to use a sliding window over ever region in the image.

For instance if the image were 
```
[[0.39, 0.51, 0.19, 0.87],
 [0.68, 0.27, 0.13, 0.24],
 [0.45, 0.73, 0.28, 0.18],
 [0.76, 0.12, 0.43, 0.19]]
```

Then we might use a 3x3 sliding window to classifying
```
[[0.39, 0.51],
 [0.68, 0.27]]
```
as `0`

```
[[ 0, '?', '?', '?'],
 ['?', '?', '?', '?'],
 ['?', '?', '?', '?'],
 ['?', '?', '?', '?']]
```
Classifying
```
[[0.39, 0.51, 0.19],
 [0.68, 0.27, 0.13]]
```
As `1`

```
[[ 0,  1, '?', '?'],
 ['?', '?', '?', '?'],
 ['?', '?', '?', '?'],
 ['?', '?', '?', '?']]
```

And so on... This is bad/ naive for 2 important reasons
1. The regions share weights. In our example we would be re-applying the same convolutional filters to 4 points `(0, 0), (0, 1), (1, 0), (0, 1)`, which is wasteful.
2. Its slow! We are operating at $O(H \cdot W \cdot N)$ complexity (where $N$ is the compelxity of making a single inference)

### A Good Approach
Our last approach worked it was just super wasteful because we computed each region independently, so what if instead we computed every region as once. How?

Pretty simple. Pass an image through a couple of convolutional layers to create a separate channel for each possible class, then take the max over the channel dimension and back propogate, averaging cross entropy across every pixel.

```
img (3,W,H) -> conv1 (D,W,H) -> conv2 (D,W,H) -> ... ->  convN(C, W, H) -> max((C,W, H)) -> (1, W, H)
```

This all seems pretty good but there is one crucial weakness. Number of parameters. Aka the network is really big.

The number of parameters in a single convolutional layer is 

$$((W \cdot H \cdot I) + 1) \cdot O$$

Where $I$ is the input number of channels and $O$ is the output

In [23]:
def good_approach_complexity(W, H, D, C, N):
    n_params = 0
    I = 3
    new_channel = [D] * (N-1) + [C]
    for O in new_channel:
        n_params += ((W * H * I) + 1) * O
        I = O
    
    s = """
    Given
        {W} x {H} image 
        {N} convolutional layers
        {D} as the intermeddiate channel size
        {C} as the number of classes
    
    There would be {n_params:,} parameters
    """.format(W=W, H=H, N=N, D=D, C=C, n_params=n_params)
    
    print(s)
    
good_approach_complexity(W=1000, H=2000, D=4, C=5, N=10)


    Given
        1000 x 2000 image 
        10 convolutional layers
        4 as the intermeddiate channel size
        5 as the number of classes
    
    There would be 320,000,041 parameters
    


### A Better Approach

So we liked the last approach its just too complex. So how do we fix it? Its pretty simple actual, we just want to reduce the number of features we are working with, so we can use downsampling to reduce the height and width of the image, then upsampling towards the end, to enlarge the image back to its original dimensions.

```
img (3, W, H) -> downsample (D, W/2, H/2) -> downsample (D, H/4, W/4) -> upsample (D, H/2, W/2) 

-> upsample (D, H, W) -> (1, W, H)
```
For downsampling techniques we can use either
- pooling
- strided convolution

And for upsampling techniques we can use
- transpose convolution

Note that this approach is called a **U-Net**

### References
- [CS 231n Lecture 11 - Detection and Segmentation](https://www.youtube.com/watch?v=nDPWywWRIRo)